# Feature Engineering

## Amaç

Bu notebookta ham veriden model performansını artıracak yeni özellikler (features) türeteceğiz.

### Yapılacak İşlemler
1. **Ratio Features**: Oransal özellikler (örn. oturum başına oyun süresi)
2. **Interaction Features**: Değişkenlerin etkileşimi
3. **Categorical Encoding**: Kategorik değişkenlerin sayısal temsili
4. **Scaling**: Özelliklerin aynı ölçeğe getirilmesi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import silhouette_score
import warnings
import sys
from pathlib import Path

# Add src to path
sys.path.append(str(Path('..').resolve()))
from src.config import *
from src.data_loader import load_gaming_dataset

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
print("Libraries imported successfully!")

## Veri Yükleme

In [ ]:
df = load_gaming_dataset(RAW_DATA_DIR)
print(f"Dataset loaded: {df.shape}")
df.head()

## 1. Ratio Features (Oransal Özellikler)

Mevcut değişkenleri birbirine oranlayarak yeni anlamlı metrikler oluşturuyoruz.

In [ ]:
# PlayTime per Session (Oturum başına oyun süresi - saat cinsinden)
# PlayTimeHours / SessionsPerWeek
# 0'a bölme hatasını önlemek için +1 eklemiyoruz çünkü SessionsPerWeek genelde > 0 ama kontrol edelim
df['playtime_per_session'] = df['PlayTimeHours'] / df['SessionsPerWeek'].replace(0, 1)

# Player Level Efficiency
# PlayerLevel / PlayTimeHours (Saat başına ne kadar seviye atlanmış)
df['level_efficiency'] = df['PlayerLevel'] / df['PlayTimeHours'].replace(0, 1)

# Achievement Rate
# AchievementUnlocked / PlayTimeHours
df['achievement_rate'] = df['AchievementUnlocked'] / df['PlayTimeHours'].replace(0, 1)

print("Ratio features created")
df[['playtime_per_session', 'level_efficiency', 'achievement_rate']].head()

## 2. Interaction Features (Etkileşim Özellikleri)

Davranışsal kalıpları yakalamak için boolean (True/False) veya segmentasyon bayrakları oluşturuyoruz.

In [ ]:
# Hardcore Gamer: Çok oynayan ve zor seviyede oynayan
# PlayTime > 75. persentil VE GameDifficulty == 'Hard'

playtime_thresh = df['PlayTimeHours'].quantile(0.75)
df['is_hardcore'] = ((df['PlayTimeHours'] > playtime_thresh) & (df['GameDifficulty'] == 'Hard')).astype(int)

# High Value Player: Satın alım yapan ve seviyesi yüksek
level_thresh = df['PlayerLevel'].quantile(0.75)
df['is_high_value'] = ((df['InGamePurchases'] == 1) & (df['PlayerLevel'] > level_thresh)).astype(int)

print("Interaction features created")
print(f"Hardcore Players: {df['is_hardcore'].sum()}")
print(f"High Value Players: {df['is_high_value'].sum()}")

## 3. Categorical Encoding

Modelin (K-Means) anlayabilmesi için metin tabanlı değişkenleri sayısal hale getiriyoruz.
- One-Hot Encoding: GameGenre, Location, GameDifficulty için kullanacağız.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_cols = ['GameGenre', 'Location', 'GameDifficulty']

# Create dummies
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True, dtype=int)

# Gender - Binary Encode
df_encoded['Gender'] = df_encoded['Gender'].map({'Male': 1, 'Female': 0}).fillna(0)

print("Categorical encoding completed")
print(f"New shape: {df_encoded.shape}")
df_encoded.head()

## 4. Etki Analizi

Yeni türetilen özelliklerin kümeleme başarısına etkisini basit bir K-Means denemesi ile görelim.

In [ ]:
# Drop non-numeric for clustering test
drop_cols = ['PlayerID', 'EngagementLevel'] # Target/ID cols
X = df_encoded.drop(columns=[c for c in drop_cols if c in df_encoded.columns])

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Simple K-Means
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_scaled)

score = silhouette_score(X_scaled, labels)
print(f"Silhouette Score with new features: {score:.4f}")

## Sonuç

Bu özellikleri `pipeline.py` içerisine entegre ederek canlı sistemde kullanabiliriz.